In [3]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline

In [11]:
df = pd.read_csv('iris.csv')
df

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SepalLength  150 non-null    float64
 1   SepalWidth   150 non-null    float64
 2   PetalLength  150 non-null    float64
 3   PetalWidth   150 non-null    float64
 4   Name         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [13]:
df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [14]:
df.shape

(150, 5)

In [15]:
df['Name'] = LabelEncoder().fit(df['Name']).transform(df['Name'])

In [16]:
df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [17]:
X = df.drop(['Name'], axis = 1)
y = df['Name']

In [18]:
sets = [train_test_split(X, y, test_size = i / 10, random_state=0) for i in range(1, 6)]

In [19]:
acc_dict = {}

models = [i for i in range(1, 6)]

acc_dict["Test Split"] = []
for i in models:
    acc_dict[f"Test Split"].append(i / 10)
    acc_dict[f"KNN {i}"] = []
acc_dict[f"Ensemble"] = []

for i in enumerate(sets):
    X_train, X_test, y_train, y_test = i[1]
    
    models = []
    
    for j in range(1, 6):
        knn = Pipeline([('m', KNeighborsClassifier(n_neighbors=j, metric='euclidean'))])
        models.append((f'knn{j}', knn))
    
    for j in models:
        y_pred = j[-1].fit(X_train, y_train).predict(X_test)
        acc_dict[f"KNN {(i[0] + 1)}"].append(metrics.accuracy_score(y_test, y_pred))
    
    ensemble = VotingClassifier(estimators = models, voting = 'soft', flatten_transform=True)
    model = ensemble.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    
    acc_dict["Ensemble"].append(metrics.accuracy_score(y_test, y_pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

In [21]:
print(acc_dict)
finaldf = pd.DataFrame(acc_dict)
finaldf.to_csv('final_iris.csv')

{'Test Split': [0.1, 0.2, 0.3, 0.4, 0.5], 'KNN 1': [1.0, 0.9333333333333333, 1.0, 1.0, 1.0], 'KNN 2': [1.0, 0.9666666666666667, 0.9666666666666667, 1.0, 0.9666666666666667], 'KNN 3': [0.9777777777777777, 0.9777777777777777, 0.9777777777777777, 0.9777777777777777, 0.9777777777777777], 'KNN 4': [0.9166666666666666, 0.8833333333333333, 0.9333333333333333, 0.9, 0.95], 'KNN 5': [0.9066666666666666, 0.8933333333333333, 0.9333333333333333, 0.92, 0.96], 'Ensemble': [1.0, 1.0, 0.9777777777777777, 0.9166666666666666, 0.9333333333333333]}
